In [24]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import simplejson as json
import geocoder
import requests
import csv
import re
import glob

In [24]:
###################################################### combine CSV data for Rivers
all_files = glob.glob("river/*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    li.append(df)

water = pd.concat(li, axis=0, ignore_index=True)

#water = pd.read_csv('river/River-2019.csv')
water = water.rename(columns={"採樣分區": "sampling_zone",
                              "河川": "river",
                              "縣市": "city",
                              "測站名稱": "station",
                              "水體分類等級": "water_body_type",
                              "採樣日期": "sampling_date",
                              "測站編號": "station_no",
                              "河川污染指數": "rpi",
                              "氣溫": "air_temp",
                              "水溫": "water_temp",
                              "酸鹼值": "PH",
                              "導電度": "conductivity",
                              "溶氧(滴定法)": "dissolved_oxygen_titration",
                              "溶氧(電極法)": "dissolved_oxygen_electrode_method",
                              "溶氧飽和度": "dissolved_oxygen_saturation",
                              "生化需氧量": "biochemical_oxygen_demand",
                              "化學需氧量": "cod",
                              "懸浮固體": "suspended_solids",
                              "大腸桿菌群": "EColi",
                              "氨氮": "ammonia_nitrogen",
                              "氯鹽": "chloride_salt",
                              "總磷": "total_phosphorus",
                              "總有機碳": "total_organic_carbon",
                              "硝酸鹽氮": "nitrate_nitrogen",
                              "亞硝酸鹽氮": "nitrite nitrogen",
                              "鎘": "cadmium",
                              "鉛": "lead",
                              "六價鉻": "hexavalent_chromium",
                              "砷": "arsenic",
                              "汞": "HG",
                              "銅": "copper",
                              "鋅": "zinc",
                              "錳": "manganese",
                              "銀": "silver",
                              "鎳": "nickel",
                              "硒": "selenium",
                              "濁度": "turbidity",
                              "鉻": "chromium",
                              "總氮": "total_nitrogen",
                              "總凱氏氮": "total_kjeldhal_nitrogen",
                              "備註": "remarks"
                              })
water["sampling_date"] = water["sampling_date"].str.replace('上午.', 'AM ', regex=True)
water["sampling_date"] = water["sampling_date"].str.replace('下午.', 'PM ', regex=True)
water['sampling_date'] = pd.to_datetime(water['sampling_date'], format='%Y/%m/%d %p %I:%M:%S')
water['sampling_date_str'] = water['sampling_date'].dt.strftime('%Y-%m-%d %I:%M:%S %p')
water['sampling_year'] = water['sampling_date'].dt.strftime('%Y') # year 
water['sampling_month_year'] = water['sampling_date'].dt.strftime('%B %Y') # month year
water = water.iloc[1:] #remove first row with data type labels
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows",10)
water

###################################################### Create a new dataframe containing only unique location information 

locations = water[['sampling_zone', 'river', 'city', 'station', 'station_no']] #remove 'water_body_type' 
locations = locations.drop_duplicates()
locations.dropna(inplace=True)
locations['city_river'] = locations['city'] + water['river']
#locations['city_river_to_review'] = locations['city_river'].str.contains(re.escape('('), na=False)
locations['lat'] = ''     #fill in station Gis_Y
locations['lon'] = ''     #fill in station Gis_X
locations['address'] = '' #fill in station address

###################################################### Fill in station location with data from https://wq.epa.gov.tw/WS_WQEPA/ws_wqepa.asmx/GetWQEPASampleData_River (waterbody_id: 1)
tree = ET.parse('stations/river_stations.xml')

for fact in tree.iter(tag = 'dt_tmp'):
    locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lat'] = fact.find('Gis_Y').text
    locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lon'] = fact.find('Gis_X').text
    if fact.find('Station_Address') != None:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'address'] = fact.find('Station_Address').text
    else:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'address'] = fact.find('Station_Name').text
'''      
with open('river_stations.json', 'w', encoding='utf-8') as file:
    locations.to_json(file, orient='records', indent=4, force_ascii=False)
''' 
###################################################### Map station information back to data
river_water_final = pd.merge(water, locations, on='station_no')
river_water_final = river_water_final.astype({'station_no': 'int32'})
with open('final/river.json', 'w', encoding='utf-8') as file:
    river_water_final.to_json(file, orient='records', indent=4, force_ascii=False)

river_water_final


,sampling_zone_x,river_x,city_x,station_x,water_body_type,sampling_date,station_no,rpi,air_temp,water_temp,PH,conductivity,dissolved_oxygen_titration,dissolved_oxygen_electrode_method,dissolved_oxygen_saturation,biochemical_oxygen_demand,cod,suspended_solids,EColi,ammonia_nitrogen,chloride_salt,total_phosphorus,total_organic_carbon,nitrate_nitrogen,nitrite nitrogen,cadmium,lead,hexavalent_chromium,arsenic,HG,copper,zinc,manganese,silver,selenium,turbidity,chromium,total_nitrogen,total_kjeldhal_nitrogen,remarks,nickel,sampling_date_str,sampling_year,sampling_month_year,sampling_zone_y,river_y,city_y,station_y,city_river,lat,lon,address
0,雙溪流域,雙溪,新北市,新寮大橋,甲,2017-12-05 06:55:00,1034,1.0,15.7,18.4,7.3,81,--,9.5,100.7,<1.0,6.7,1.6,6000,0.07,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,NaN,2017-12-05 06:55:00 AM,2017,December 2017,雙溪流域,雙溪,新北市,新寮大橋,新北市雙溪,25.0317969,121.8347690,新北市雙溪區上林村北38號道
1,雙溪流域,雙溪,新北市,新寮大橋,甲,2017-11-07 07:55:00,1034,1.0,25.6,22.6,7.4,88,--,9.0,103.4,1.3,5.9,1.6,7000,0.04,--,0.020,--,0.59,--,<0.001,<0.003,<0.002,<0.0003,<0.0003,<0.001,0.016,0.010,<0.001,--,--,--,--,--,NaN,NaN,2017-11-07 07:55:00 AM,2017,November 2017,雙溪流域,雙溪,新北市,新寮大橋,新北市雙溪,25.0317969,121.8347690,新北市雙溪區上林村北38號道
2,雙溪流域,雙溪,新北市,新寮大橋,甲,2017-10-12 07:55:00,1034,1.0,26.4,23.6,7.4,82,--,8.8,104.8,<1.0,7.1,12.8,17000,0.03,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,NaN,2017-10-12 07:55:00 AM,2017,October 2017,雙溪流域,雙溪,新北市,新寮大橋,新北市雙溪,25.0317969,121.8347690,新北市雙溪區上林村北38號道
3,雙溪流域,雙溪,新北市,新寮大橋,甲,2017-09-06 07:55:00,1034,1.0,30.1,25.6,7.5,89,--,8.4,103.9,<1.0,5.5,<1.0,3700,<0.01,--,0.021,--,0.51,--,<0.001,<0.003,<0.002,<0.0003,<0.0003,<0.001,0.015,0.013,<0.001,--,--,--,--,--,NaN,NaN,2017-09-06 07:55:00 AM,2017,September 2017,雙溪流域,雙溪,新北市,新寮大橋,新北市雙溪,25.0317969,121.8347690,新北市雙溪區上林村北38號道
4,雙溪流域,雙溪,新北市,新寮大橋,甲,2017-08-08 07:55:00,1034,1.0,30.0,30.2,7.3,129,--,7.3,97.8,<1.0,7.6,3.6,2900,0.24,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,NaN,2017-08-08 07:55:00 AM,2017,August 2017,雙溪流域,雙溪,新北市,新寮大橋,新北市雙溪,25.0317969,121.8347690,新北市雙溪區上林村北38號道
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10899,南崁溪流域,南崁溪,桃園市,天助橋,丙,2019-03-05 13:25:00,1674,4.5,25.0,23.4,7.76,1630,--,8.4,99.4,9.5,24.1,11.6,69000,4.51,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,--,2019-03-05 01:25:00 PM,2019,March 2019,南崁溪流域,南崁溪,桃園市,天助橋,桃園市南崁溪,25.0070278,121.3141111,桃園市桃園區民光東路附近
10900,南崁溪流域,南崁溪,桃園市,天助橋,丙,2019-02-13 13:27:00,1674,6.0,22.0,21.5,7.20,1560,--,7.0,79.4,20.4,35.3,26.3,150000,5.81,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,--,2019-02-13 01:27:00 PM,2019,February 2019,南崁溪流域,南崁溪,桃園市,天助橋,桃園市南崁溪,25.0070278,121.3141111,桃園市桃園區民光東路附近
10901,南崁溪流域,南崁溪,桃園市,天助橋,丙,2019-01-04 13:26:00,1674,4.3,26.0,24.8,7.71,1280,--,7.6,90.9,4.3,32.7,20.3,71000,4.93,--,1.96,6.13,6.05,0.988,<0.001,<0.003,<0.002,0.0045,<0.0003,0.012,0.049,0.078,<0.001,<0.001,--,--,--,--,NaN,0.007,2019-01-04 01:26:00 PM,2019,January 2019,南崁溪流域,南崁溪,桃園市,天助橋,桃園市南崁溪,25.0070278,121.3141111,桃園市桃園區民光東路附近
10902,後龍溪流域,後龍溪,苗栗縣,明德四號橋,乙,2019-08-15 12:00:00,1704,1.0,29.4,27.4,8.97,206,--,8.7,112.8,2.5,12.0,8.0,210,0.12,--,0.020,1.89,0.09,0.010,--,--,--,--,--,--,--,--,--,--,--,--,0.22,--,NaN,--,2019-08-15 12:00:00 PM,2019,August 2019,後龍溪流域,後龍溪,苗栗縣,明德四號橋,苗栗縣後龍溪,24.5886111,120.9189444,明德四號橋


In [29]:
'''
Perform River stats

https://medium.com/dunder-data/finding-the-percentage-of-missing-values-in-a-pandas-dataframe-a04fa00f84ab
https://queirozf.com/entries/pandas-dataframe-groupby-examples
'''

YEAR = '2018'
FIELD = 'rpi'
FIELD_MISSING = FIELD +'_missing'
MISSING_VALUE = '--'

# RPI stats - record min, max, average, number of missing data for GIVEN YEAR at each station, use station 1234 for testing
river_stats = river_water_final[river_water_final['sampling_year'] == YEAR]
river_stats[FIELD] = river_stats[FIELD].replace(MISSING_VALUE, np.nan)

# Remove rows with np.nan in rpi column for GIVEN YEAR
#river_water_finalRPI = river_water_finalRPI.dropna(subset=[FIELD])

river_stats = river_stats.astype({FIELD: 'float'})
river_stats[FIELD_MISSING] = river_stats[FIELD].isna()
river_stats = river_stats.groupby(['sampling_year', 'station_no', 'address', 'lat', 'lon']).agg(
    {
     FIELD: ['min', 'max', 'mean', 'count'],
     FIELD_MISSING: ['sum', 'mean']
    }
) #.rename(columns={"min": "RPI_min", "max": "RPI_max", "mean": "RPI_mean"})

# rename columns 
river_stats.columns = ['_'.join(col).strip() for col in river_stats.columns.values]

# format values
river_stats = river_stats.astype({FIELD_MISSING + '_sum': 'int32'})
river_stats[FIELD_MISSING + '_mean'] = river_stats[FIELD_MISSING + '_mean'].round(4) * 100
river_stats[FIELD + '_mean'] = river_stats[FIELD + '_mean'].round(2)

#rename columns so as to easier handle different stat types
river_stats.rename(columns={
    FIELD + '_min': "min",
    FIELD + '_max': "max",
    FIELD + '_mean': "mean",
    FIELD + '_count': "count",
    FIELD_MISSING + '_sum': "num_missing",
    FIELD_MISSING + '_mean': "perc_missing",
}, inplace=True)
river_stats = river_stats.reset_index()
#river_water_final2['river_pollution_index'].unique()

#river_stats['min'].dropna(inplace=True)
river_stats = river_stats.dropna(how='any', subset=['min'])
river_stats[river_stats['station_no'] == 1095]

# 1143 is missing all data, remove missing stations from analysis 
#river_stats

c:\users\user\envs\wata\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,sampling_year,station_no,address,lat,lon,min,max,mean,count,num_missing,perc_missing
81,2018,1095,苗栗縣頭份鎮尖山台1號與13號公路交叉口,24.6669460,120.8817000,1.0,5.5,3.94,12,0,0.0


In [7]:
###################################################### combine CSV data for DAM
all_files = glob.glob("dam/*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    li.append(df)

dam = pd.concat(li, axis=0, ignore_index=True)

dam = dam.rename(columns={"採樣分區": "sampling_zone",
                              "縣市": "city",
                              "測站名稱": "station",
                              "水體分類等級": "water_body_type",
                              "採樣日期": "sampling_date",
                              "測站編號": "station_no",
                              "卡爾森指數": "ctsi",
                              "採樣深度": "sampling_depth",
                              "氣溫": "air_temp",
                              "水溫": "water_temp",
                              "酸鹼值": "PH",
                              "導電度": "conductivity",
                              "溶氧(滴定法)": "dissolved_oxygen_titration",
                              "溶氧(電極法)": "dissolved_oxygen_electrode_method",
                              "溶氧飽和度": "dissolved_oxygen_saturation",
                              "透明度": "transparency",
                              "葉綠素a": "Chlorophylla",
                              "正磷酸鹽": "Orthophosphate",
                              "濁度": "turbidity",
                              "懸浮固體": "suspended_solids",
                              "化學需氧量": "cod",
                              "氨氮": "ammonia_nitrogen",
                              "總硬度": "total_hardness",
                              "總鹼度": "total_alkalinity",
                              "總磷": "total_phosphorus",
                              "硝酸鹽氮": "nitrate_nitrogen",
                              "亞硝酸鹽氮": "nitrite nitrogen",
                              "總有機碳": "total_organic_carbon",
                              "有機氮": "organic_nitrogen",
                              "總凱氏氮": "total_kjeldhal_nitrogen",
                              "備註": "remarks"                             
                              })
dam["sampling_date"] = dam["sampling_date"].str.replace('上午.', 'AM ', regex=True)
dam["sampling_date"] = dam["sampling_date"].str.replace('下午.', 'PM ', regex=True)
dam['sampling_date'] = pd.to_datetime(dam['sampling_date'], format='%Y/%m/%d %p %I:%M:%S')
dam['sampling_date_str'] = dam['sampling_date'].dt.strftime('%Y-%m-%d %I:%M:%S %p')
dam['sampling_year'] = dam['sampling_date'].dt.strftime('%Y') # year 
dam['sampling_month_year'] = dam['sampling_date'].dt.strftime('%B %Y') # month year
dam = dam.iloc[1:] #remove first row with data type labels
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows",10)
dam

###################################################### Create a new dataframe containing only unique location information 

locations = dam[['sampling_zone', 'city', 'station', 'station_no']] #remove 'water_body_type' 
locations = locations.drop_duplicates()
locations.dropna(inplace=True)
locations['lat'] = ''     #fill in station Gis_Y
locations['lon'] = ''     #fill in station Gis_X
locations['address'] = '' #fill in station address

###################################################### Fill in station location with data from https://wq.epa.gov.tw/WS_WQEPA/ws_wqepa.asmx/GetWQEPASampleData_River (waterbody_id: 1)
tree = ET.parse('stations/reservoir_stations.xml')

for fact in tree.iter(tag = 'dt_tmp'):
    if fact.find('Gis_Y') != None:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lat'] = fact.find('Gis_Y').text
    else:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lat'] = "none"
        print(fact.find('Station_ID1').text + " has no GIS_Y" )
        
    if fact.find('Gis_X') != None:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lon'] = fact.find('Gis_X').text
    else:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lon'] = "none"
        print(fact.find('Station_ID1').text + " has no Gis_X")
        
    if fact.find('Station_Address') != None:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'address'] = fact.find('Station_Address').text
    else:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'address'] = fact.find('Station_Name').text
'''      
with open('river_stations.json', 'w', encoding='utf-8') as file:
    locations.to_json(file, orient='records', indent=4, force_ascii=False)
''' 
###################################################### Map station information back to data
dam_water_final = pd.merge(dam, locations, on='station_no')
dam_water_final = dam_water_final.astype({'station_no': 'int32'})
with open('final/dam.json', 'w', encoding='utf-8') as file:
    dam_water_final.to_json(file, orient='records', indent=4, force_ascii=False)

dam_water_final  
#2123


2198 has no GIS_Y
2198 has no Gis_X
2199 has no GIS_Y
2199 has no Gis_X
2200 has no GIS_Y
2200 has no Gis_X


,sampling_zone_x,city_x,station_x,water_body_type,sampling_date,station_no,ctsi,sampling_depth,air_temp,water_temp,PH,conductivity,dissolved_oxygen_titration,dissolved_oxygen_electrode_method,dissolved_oxygen_saturation,transparency,Chlorophylla,total_phosphorus,Orthophosphate,turbidity,suspended_solids,cod,ammonia_nitrogen,total_hardness,total_alkalinity,nitrate_nitrogen,nitrite nitrogen,total_organic_carbon,organic_nitrogen,total_kjeldhal_nitrogen,remarks,sampling_date_str,sampling_year,sampling_month_year,sampling_zone_y,city_y,station_y,lat,lon,address
0,阿公店水庫,高雄市,阿公店水庫一,甲,2017-11-15 10:02:00,2075,51,0.5,27.5,26.8,8.2,449,--,7.8,97.8,0.6,1.7,0.023,--,3.7,2.9,6.5,<0.01,--,--,--,--,--,--,--,NaN,2017-11-15 10:02:00 AM,2017,November 2017,阿公店水庫,高雄市,阿公店水庫一,22.8151120,120.3442810,高雄市燕巢區
1,阿公店水庫,高雄市,阿公店水庫一,甲,2017-09-12 10:38:00,2075,54,0.5,30.6,31.4,8.5,415,--,6.8,92.9,0.8,1.8,0.052,--,10,8.7,5.9,0.02,--,--,--,--,--,--,--,NaN,2017-09-12 10:38:00 AM,2017,September 2017,阿公店水庫,高雄市,阿公店水庫一,22.8151120,120.3442810,高雄市燕巢區
2,阿公店水庫,高雄市,阿公店水庫一,甲,2017-05-19 09:57:00,2075,66,0.5,32.5,28.2,8.4,467,--,7.6,100.1,0.6,32.4,0.066,--,50,39.6,10.4,0.06,--,--,--,--,--,--,--,NaN,2017-05-19 09:57:00 AM,2017,May 2017,阿公店水庫,高雄市,阿公店水庫一,22.8151120,120.3442810,高雄市燕巢區
3,阿公店水庫,高雄市,阿公店水庫一,甲,2017-02-16 10:30:00,2075,51,0.5,24.8,21.8,8.3,447,--,8.7,98.5,0.8,2.6,0.026,--,8.4,7.0,5.6,<0.01,--,--,--,--,--,--,--,NaN,2017-02-16 10:30:00 AM,2017,February 2017,阿公店水庫,高雄市,阿公店水庫一,22.8151120,120.3442810,高雄市燕巢區
4,阿公店水庫,高雄市,阿公店水庫一,乙,2018-12-18 13:55:00,2075,54,0.5,27.3,24.1,8.28,426,--,8.4,99.1,0.7,4.6,0.029,--,14,11.8,5.4,<0.01,--,--,--,--,2.06,--,--,NaN,2018-12-18 01:55:00 PM,2018,December 2018,阿公店水庫,高雄市,阿公店水庫一,22.8151120,120.3442810,高雄市燕巢區
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4347,湖山水庫,雲林縣,湖山水庫,乙,2019-05-09 13:00:00,2210,49,0.5,24.5,25.6,8.44,392,--,8.7,110.0,1.2,4.0,0.016,--,5.1,5.7,5.8,0.02,--,--,--,--,1.49,--,--,NaN,2019-05-09 01:00:00 PM,2019,May 2019,湖山水庫,雲林縣,湖山水庫,23.6840833,120.6255556,雲林縣斗六市及古坑鄉
4348,湖山水庫,雲林縣,湖山水庫,乙,2019-04-15 10:30:00,2210,42,0.5,24.3,26.5,8.48,385,--,8.6,108.8,1.6,2.8,0.007,--,2.7,4.4,5.8,0.02,--,--,--,--,1.53,--,--,NaN,2019-04-15 10:30:00 AM,2019,April 2019,湖山水庫,雲林縣,湖山水庫,23.6840833,120.6255556,雲林縣斗六市及古坑鄉
4349,湖山水庫,雲林縣,湖山水庫,乙,2019-03-11 09:00:00,2210,42,0.5,18.6,20.9,8.00,375,--,5.0,57.4,2.0,5.3,0.006,--,3.0,3.0,4.8,0.02,--,--,--,--,1.37,--,--,NaN,2019-03-11 09:00:00 AM,2019,March 2019,湖山水庫,雲林縣,湖山水庫,23.6840833,120.6255556,雲林縣斗六市及古坑鄉
4350,湖山水庫,雲林縣,湖山水庫,乙,2019-02-18 09:15:00,2210,41,0.5,21.5,22.8,8.32,371,--,8.8,103.6,3.0,6.0,0.006,--,2.2,2.3,<4.0,0.03,--,--,--,--,1.41,--,--,NaN,2019-02-18 09:15:00 AM,2019,February 2019,湖山水庫,雲林縣,湖山水庫,23.6840833,120.6255556,雲林縣斗六市及古坑鄉


In [17]:
'''
Perform DAM stats

https://medium.com/dunder-data/finding-the-percentage-of-missing-values-in-a-pandas-dataframe-a04fa00f84ab
https://queirozf.com/entries/pandas-dataframe-groupby-examples
'''

YEAR = '2017'
FIELD = 'ctsi'
FIELD_MISSING = FIELD +'_missing'
MISSING_VALUE = '--'

# field stats - record min, max, average, number of missing data for GIVEN YEAR at each station, use station 1234 for testing
dam_stats = dam_water_final[dam_water_final['sampling_year'] == YEAR]
dam_stats[FIELD] = dam_water_final[FIELD].replace(MISSING_VALUE, np.nan)

# Remove rows with np.nan in rpi column for GIVEN YEAR
#river_water_finalRPI = river_water_finalRPI.dropna(subset=[FIELD])

dam_stats = dam_stats.astype({FIELD: 'float'})
dam_stats[FIELD_MISSING] = dam_stats[FIELD].isna()
dam_stats = dam_stats.groupby(['sampling_year', 'station_no', 'address', 'lat', 'lon']).agg(
    {
     FIELD: ['min', 'max', 'mean', 'count'],
     FIELD_MISSING: ['sum', 'mean']
    }
) #.rename(columns={"min": "RPI_min", "max": "RPI_max", "mean": "RPI_mean"})

# rename columns 
dam_stats.columns = ['_'.join(col).strip() for col in dam_stats.columns.values]

# format values
dam_stats = dam_stats.astype({FIELD_MISSING + '_sum': 'int32'})
dam_stats[FIELD_MISSING + '_mean'] = dam_stats[FIELD_MISSING + '_mean'].round(4) * 100
dam_stats[FIELD + '_mean'] = dam_stats[FIELD + '_mean'].round(2)

#rename columns so as to easier handle different stat types
dam_stats.rename(columns={
    FIELD + '_min': "min",
    FIELD + '_max': "max",
    FIELD + '_mean': "mean",
    FIELD + '_count': "count",
    FIELD_MISSING + '_sum': "num_missing",
    FIELD_MISSING + '_mean': "perc_missing",
}, inplace=True)
dam_stats = dam_stats.reset_index()
#river_water_final2['river_pollution_index'].unique()

#river_stats['min'].dropna(inplace=True)
dam_stats = dam_stats.dropna(how='any', subset=['min'])
dam_stats[dam_stats['station_no'] == 2152]

# 1143 is missing all data, remove missing stations from analysis 
#river_stats

c:\users\user\envs\wata\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,sampling_year,station_no,address,lat,lon,min,max,mean,count,num_missing,perc_missing
53,2017,2152,苗栗縣頭屋鄉,24.5804260,120.8847610,47.0,59.0,53.58,12,24,66.67


In [23]:
###################################################### combine CSV data for GROUND WATER
all_files = glob.glob("gd_water/*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    li.append(df)

gdwater = pd.concat(li, axis=0, ignore_index=True)

gdwater = gdwater.rename(columns={"採樣分區": "sampling_zone",
                              "縣市": "city",
                              "測站名稱": "station",
                              "水體分類等級": "water_body_type",
                              "採樣日期": "sampling_date",
                              "測站編號": "station_no",
                              "水面至井口深度": "water_surface_to_wellhead_depth",
                              "水溫": "water_temp",
                              "酸鹼值": "PH",
                              "導電度": "conductivity",
                              "溶氧(電極法)": "dissolved_oxygen_electrode_method",
                              "溶氧飽和度": "dissolved_oxygen_saturation",
                              "總硬度": "total_hardness",
                              "總溶解固體物": "total_dissolved_solids",
                              "氯鹽": "chloride_salt",
                              "氨氮": "ammonia_nitrogen",
                              "硝酸鹽氮": "nitrate_nitrogen",
                              "硫酸鹽": "sulfate",
                              "總有機碳": "total_organic_carbon",
                              "砷": "arsenic",
                              "鎘": "cadmium",
                              "鉻": "chromium",
                              "銅": "copper",
                              "鉛": "lead",
                              "鋅": "zinc",
                              "鐵": "iron",
                              "錳": "manganese",
                              "汞": "HG",
                              "鎳": "nickel",
                              "總酚": "total_phenol",
                              "氟鹽": "fluoride_salt",
                              "鈉": "sodium",
                              "鉀": "potassium",
                              "鈣": "calcium",
                              "鎂": "magnesium",
                              "總鹼度": "total_alkinity",
                              "苯": "benzene",
                              "甲苯": "toluene",
                              "乙苯": "ethylbenzene",
                              "二甲苯": "xylene",
                              "四氯化碳": "carbon_tetrachloride",
                              "氯苯": "chlorobenzene",
                              "氯仿": "chloroform",
                              "氯甲烷": "methyl_chloride",
                              "1，4—二氯苯": "1-4-dichlorobenzene",
                              "1，1—二氯乙烷": "1-1-dichloroethane",
                              "1，2—二氯乙烷": "1-2-dichloroethane",
                              "1，1—二氯乙烯": "1-1-dichloroethylene",
                              "順1，2—二氯乙烯": "cis1-2-dichloroethylene",
                              "反1，2—二氯乙烯": "trans1-2-dichloroethylene",
                              "三氯乙烯": "trichloroethylene",
                              "四氯乙烯": "tetrachloroethylene",
                              "氯乙烯": "vinyl_chloride",
                              "二氯甲烷": "methylene_chloride",
                              "1，1，2—三氯乙烷": "1-1-2-trichloroethane",
                              "1，1，1—三氯乙烷": "1-1-1-trichloroethane",
                              "1，2-二氯苯": "1-2-dichlorobenzene",
                              "甲基第三丁基醚": "methyl_tertiary_butyl_ether",
                              "備註": "remarks"                             
                              })
gdwater["sampling_date"] = gdwater["sampling_date"].str.replace('上午.', 'AM ', regex=True)
gdwater["sampling_date"] = gdwater["sampling_date"].str.replace('下午.', 'PM ', regex=True)
gdwater['sampling_date'] = pd.to_datetime(gdwater['sampling_date'], format='%Y/%m/%d %p %I:%M:%S')
gdwater['sampling_date_str'] = gdwater['sampling_date'].dt.strftime('%Y-%m-%d %I:%M:%S %p')
gdwater['sampling_year'] = gdwater['sampling_date'].dt.strftime('%Y') # year 
gdwater['sampling_month_year'] = gdwater['sampling_date'].dt.strftime('%B %Y') # month year
gdwater = gdwater.iloc[1:] #remove first row with data type labels
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows",10)
gdwater

###################################################### Create a new dataframe containing only unique location information 

locations = gdwater[['sampling_zone', 'city', 'station', 'station_no']] #remove 'water_body_type' 
locations = locations.drop_duplicates()
locations.dropna(inplace=True)
locations['lat'] = ''     #fill in station Gis_Y
locations['lon'] = ''     #fill in station Gis_X
locations['address'] = '' #fill in station address

###################################################### Fill in station location with data from https://wq.epa.gov.tw/WS_WQEPA/ws_wqepa.asmx/GetWQEPASampleData_River (waterbody_id: 1)
tree = ET.parse('stations/groundwater_stations.xml')

for fact in tree.iter(tag = 'dt_tmp'):
    if fact.find('Gis_Y') != None:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lat'] = fact.find('Gis_Y').text
    else:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lat'] = "none"
        print(fact.find('Station_ID1').text + " has no GIS_Y" )
        
    if fact.find('Gis_X') != None:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lon'] = fact.find('Gis_X').text
    else:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lon'] = "none"
        print(fact.find('Station_ID1').text + " has no Gis_X")
        
    if fact.find('Station_Address') != None:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'address'] = fact.find('Station_Address').text
    else:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'address'] = fact.find('Station_Name').text
'''      
with open('river_stations.json', 'w', encoding='utf-8') as file:
    locations.to_json(file, orient='records', indent=4, force_ascii=False)
''' 
###################################################### Map station information back to data
gd_water_final = pd.merge(gdwater, locations, on='station_no')
gd_water_final = gd_water_final.astype({'station_no': 'int32'})
with open('final/gd_water.json', 'w', encoding='utf-8') as file:
    gd_water_final.to_json(file, orient='records', indent=4, force_ascii=False)

gd_water_final


4000 has no GIS_Y
4000 has no Gis_X
4001 has no GIS_Y
4001 has no Gis_X
4002 has no GIS_Y
4002 has no Gis_X
4003 has no GIS_Y
4003 has no Gis_X
4004 has no GIS_Y
4004 has no Gis_X
4005 has no GIS_Y
4005 has no Gis_X
4006 has no GIS_Y
4006 has no Gis_X
4007 has no GIS_Y
4007 has no Gis_X
4008 has no GIS_Y
4008 has no Gis_X
4009 has no GIS_Y
4009 has no Gis_X
4010 has no GIS_Y
4010 has no Gis_X
4011 has no GIS_Y
4011 has no Gis_X
4012 has no GIS_Y
4012 has no Gis_X
4013 has no GIS_Y
4013 has no Gis_X
4014 has no GIS_Y
4014 has no Gis_X
4015 has no GIS_Y
4015 has no Gis_X
4016 has no GIS_Y
4016 has no Gis_X
4017 has no GIS_Y
4017 has no Gis_X
4018 has no GIS_Y
4018 has no Gis_X
4019 has no GIS_Y
4019 has no Gis_X
4020 has no GIS_Y
4020 has no Gis_X
4021 has no GIS_Y
4021 has no Gis_X
4022 has no GIS_Y
4022 has no Gis_X
4023 has no GIS_Y
4023 has no Gis_X
4024 has no GIS_Y
4024 has no Gis_X
4025 has no GIS_Y
4025 has no Gis_X
4026 has no GIS_Y
4026 has no Gis_X
4027 has no GIS_Y
4027 has n

,sampling_zone_x,city_x,station_x,sampling_date,station_no,water_surface_to_wellhead_depth,water_temp,conductivity,PH,dissolved_oxygen_electrode_method,dissolved_oxygen_saturation,氧化還原電位,total_hardness,total_dissolved_solids,chloride_salt,ammonia_nitrogen,nitrate_nitrogen,sulfate,total_organic_carbon,arsenic,cadmium,chromium,copper,lead,zinc,iron,manganese,HG,nickel,total_phenol,fluoride_salt,sodium,potassium,calcium,magnesium,total_alkinity,benzene,toluene,ethylbenzene,xylene,naphthalene,carbon_tetrachloride,chlorobenzene,chloroform,methyl_chloride,1,4—二氯苯,1.1,1—二氯乙烷,1.2,2—二氯乙烷,1.3,1—二氯乙烯,順1,2—二氯乙烯,反1,2—二氯乙烯.1,trichloroethylene,tetrachloroethylene,vinyl_chloride,methylene_chloride,1.4,1.5,2—三氯乙烷,1.6,1.7,1—三氯乙烷,1.8,2-二氯苯,methyl_tertiary_butyl_ether,remarks,1-4-dichlorobenzene,1-1-dichloroethane,1-2-dichloroethane,1-1-dichloroethylene,cis1-2-dichloroethylene,trans1-2-dichloroethylene,1-1-2-trichloroethane,1-1-1-trichloroethane,1-2-dichlorobenzene,sampling_date_str,sampling_year,sampling_month_year,sampling_zone_y,city_y,station_y,lat,lon,address
0,臺北盆地,基隆市,碇內國中,2017-12-01 08:33:00,4145,3.922,24.4,366,5.6,1.0,--,123,132,241,17.1,0.24,0.68,102,<0.10,<0.0003,<0.001,0.001,0.002,0.003,0.008,2.18,1.19,--,--,<0.0060,<0.10,25.1,4.53,33.4,15.1,39.4,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-01 08:33:00 AM,2017,December 2017,臺北盆地,基隆市,碇內國中,25.0960530,121.7489070,基隆市暖暖區源遠路152巷75號
1,臺北盆地,基隆市,碇內國中,2017-08-17 08:18:00,4145,7.681,23.5,237,5.2,0.6,--,228,71.8,121,20.7,0.05,0.61,69.2,0.56,<0.0003,<0.001,0.002,0.001,0.003,0.006,0.021,0.323,--,--,<0.0060,<0.10,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-17 08:18:00 AM,2017,August 2017,臺北盆地,基隆市,碇內國中,25.0960530,121.7489070,基隆市暖暖區源遠路152巷75號
2,臺北盆地,基隆市,碇內國中,2017-05-09 08:29:00,4145,4.865,23.0,192,5.8,2.0,--,238,50.9,145,18.3,0.01,0.61,33.8,0.47,<0.0003,<0.001,0.002,0.003,<0.003,0.014,0.056,0.232,<0.0003,<0.005,<0.0060,0.15,12.6,2.21,13.7,4.90,24.4,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-05-09 08:29:00 AM,2017,May 2017,臺北盆地,基隆市,碇內國中,25.0960530,121.7489070,基隆市暖暖區源遠路152巷75號
3,臺北盆地,基隆市,碇內國中,2017-04-27 12:52:00,4145,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,<0.00200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-04-27 12:52:00 PM,2017,April 2017,臺北盆地,基隆市,碇內國中,25.0960530,121.7489070,基隆市暖暖區源遠路152巷75號
4,臺北盆地,基隆市,碇內國中,2017-02-18 11:24:00,4145,5.072,24.2,202,5.7,1.4,--,246,41.0,135,17.7,0.27,1.23,35.9,0.39,<0.0003,<0.001,<0.001,<0.001,<0.003,0.006,0.086,0.145,<0.0003,<0.005,<0.0060,<0.10,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-18 11:24:00 AM,2017,February 2017,臺北盆地,基隆市,碇內國中,25.0960530,121.7489070,基隆市暖暖區源遠路152巷75號
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,蘭陽平原,宜蘭縣,內城社區,2019-11-01 07:55:00,4167,1.683,23.0,226,6.96,4.0,--,NaN,102,164,3.5,<0.01,0.93,16.8,0.26,<0.0003,<0.001,<0.001,<0.001,<0.003,0.002,0.030,<0.005,--,--,<0.0060,0.10,5.49,0.913,29.1,8.33,92.1,--,--,--,--,--,--,--,--,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [22]:
'''
Perform GROUND WATER stats

https://medium.com/dunder-data/finding-the-percentage-of-missing-values-in-a-pandas-dataframe-a04fa00f84ab
https://queirozf.com/entries/pandas-dataframe-groupby-examples
'''

YEAR = '2019'
FIELD = 'water_surface_to_wellhead_depth'
FIELD_MISSING = FIELD +'_missing'
MISSING_VALUE = '--'

# field stats - record min, max, average, number of missing data for GIVEN YEAR at each station, use station 1234 for testing
gd_stats = gd_water_final[gd_water_final['sampling_year'] == YEAR]
gd_stats[FIELD] = gd_stats[FIELD].replace(MISSING_VALUE, np.nan)

# Remove rows with np.nan in rpi column for GIVEN YEAR
#river_water_finalRPI = river_water_finalRPI.dropna(subset=[FIELD])

gd_stats = gd_stats.astype({FIELD: 'float'})
gd_stats[FIELD_MISSING] = gd_stats[FIELD].isna()
gd_stats = gd_stats.groupby(['sampling_year', 'station_no', 'address', 'lat', 'lon']).agg(
    {
     FIELD: ['min', 'max', 'mean', 'count'],
     FIELD_MISSING: ['sum', 'mean']
    }
) #.rename(columns={"min": "RPI_min", "max": "RPI_max", "mean": "RPI_mean"})

# rename columns 
gd_stats.columns = ['_'.join(col).strip() for col in gd_stats.columns.values]

# format values
gd_stats = gd_stats.astype({FIELD_MISSING + '_sum': 'int32'})
gd_stats[FIELD_MISSING + '_mean'] = gd_stats[FIELD_MISSING + '_mean'].round(4) * 100
gd_stats[FIELD + '_mean'] = gd_stats[FIELD + '_mean'].round(2)

#rename columns so as to easier handle different stat types
gd_stats.rename(columns={
    FIELD + '_min': "min",
    FIELD + '_max': "max",
    FIELD + '_mean': "mean",
    FIELD + '_count': "count",
    FIELD_MISSING + '_sum': "num_missing",
    FIELD_MISSING + '_mean': "perc_missing",
}, inplace=True)
gd_stats = gd_stats.reset_index()
#river_water_final2['river_pollution_index'].unique()

#river_stats['min'].dropna(inplace=True)
gd_stats = gd_stats.dropna(how='any', subset=['min'])
gd_stats[gd_stats['station_no'] == 4145]

# 1143 is missing all data, remove missing stations from analysis 
#river_stats

c:\users\user\envs\wata\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,sampling_year,station_no,address,lat,lon,min,max,mean,count,num_missing,perc_missing
0,2019,4145,基隆市暖暖區源遠路152巷75號,25.0960530,121.7489070,4.097,6.064,5.21,4,1,20.0


In [4]:
'''
Cleanup city_river river names in locations dataframe
We are assuming that stations will fall along river borders, we can use station locations to confirm that river locations are mapped 
'''

locations = locations.append(
    [{'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'老街溪, Dayuan District, Taoyuan, 33756, Taiwan','lat':'None',
                              'lon':'None', 'address': 'None'
    },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'老街溪, Longtan District, Taoyuan, 325, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
    },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'Laojie River, Longtan District, Taoyuan, 325, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
                             },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'茄苳溪, Bade District, Taoyuan, 33058, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
                             },
     
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'茄苳溪, Bade District, Taoyuan, 33453, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
                             },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'大坑崁溪, Pingzhen District, Taoyuan, 32559, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'石門大圳, Pingzhen District, Taoyuan, 33551, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'霄裡溪, Xinpu, Hsinchu County, Taiwan Province, 305, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'上坪溪, Zhudong, Hsinchu County, Taiwan Province, 311, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'南港溪, Zaoqiao, Miaoli County, Taiwan Province, 361, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'老庄溪, Miaoli County, Taiwan Province, 369, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪, Hanxi Village, East District, Taichung, 401, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪排水, Dali District, Taichung, 41473, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'南投貓羅溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪排水, Dali District, Taichung, 41283, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪, Dunnan Village, Houli District, Taichung, 42147, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪, Xintian Village, Tanzi District, Taichung, 40647, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪, Wuri District, Taichung, 414, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'谷溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旗山溪, Jiaxian District, Kaohsiung, 847, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'東港溪, Neipu, Pingtung County, Taiwan Province, 912, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'牛稠溪, Pingtung County, Taiwan Province, 90049, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'Taiping River, Tai''an Village, Beinan, Taitung County, Taiwan Province, 954, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'Luye River, Taoyuan Village, Yanping, Taitung County, Taiwan Province, 953, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'花蓮太平溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'Lele River, Zhuoxi, Hualien County, Taiwan Province, 982, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'荖溪, Shoufeng, Hualien County, Taiwan Province, 974, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'壽豐溪(知亞干溪), Wanrong, Hualien County, Taiwan Province, 979, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'七家灣溪, Heping District, Taichung, 42413, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'南澳南溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'林尾溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     }
    ], ignore_index=True)

#TAIPEI
locations.loc[locations['city_river'] == '臺北市淡水河本流', 'city_river'] = '新北市淡水河'

#NEW TAIPEI
locations.loc[locations['city_river'] == '新北市淡水河本流', 'city_river'] = '新北市淡水河'
locations.loc[locations['city_river'] == '新北市新店溪', 'city_river'] = '臺北市新店溪'
locations.loc[locations['city_river'] == '新北市景美溪', 'city_river'] = '臺北市景美溪'
locations.loc[locations['city_river'] == '新北市基隆河', 'city_river'] = '基隆市基隆河'

#TAOYUAN
locations.loc[locations['city_river'] == '桃園市福興溪', 'city_river'] = '新竹縣福興溪'
locations.loc[locations['city_river'] == '桃園市老街溪', 'city_river'] = '中壢市老街溪'
locations.loc[locations['city_river'] == '桃園市茄苳溪(桃)', 'city_river'] = '茄苳溪, Taoyuan, Taoyuan District, Taoyuan, 330, Taiwan'
locations.loc[locations['city_river'] == '桃園市南崁溪', 'city_river'] = 'Nankan River, Guishan District, Taoyuan, 333, Taiwan'


#HSINCHU
locations.loc[locations['city_river'] == '新竹市客雅溪', 'city_river'] = 'keya creek hsinchu'
locations.loc[locations['city_river'] == '新竹縣鹽港溪', 'city_river'] = '新竹市鹽港溪'
locations.loc[locations['city_river'] == '新竹縣客雅溪', 'city_river'] = 'keya creek hsinchu'

#MIAOLI
locations.loc[locations['city_river'] == '苗栗縣南港溪(苗)', 'city_river'] = '苗栗縣中港溪'

#TAICHUNG
locations.loc[locations['city_river'] == '臺中市烏溪', 'city_river'] = '旱溪排水大里 41473'
locations.loc[locations['city_river'] == '臺中市大安溪', 'city_river'] = '苗栗縣大安溪'
locations.loc[locations['city_river'] == '彰化縣烏溪', 'city_river'] = '臺中市大里溪'

#NANTOU
locations.loc[locations['city_river'] == '南投縣清水溪(投)', 'city_river'] = '清水溪, Nantou County, Taiwan Province, 55759, Taiwan'
locations.loc[locations['city_river'] == '南投縣北港溪(投)', 'city_river'] = '南港溪, Nantou County, Taiwan Province, 544, Taiwan'

#YUNLIN
locations.loc[locations['city_river'] == '雲林縣北港溪(雲)', 'city_river'] = '雲林縣北港溪'

#HUALIEN
locations.loc[locations['city_river'] == '花蓮縣吉安溪', 'city_river'] = '吉安溪 (七腳川溪), Hualien County, Taiwan Province, 97342, Taiwan'


locations

,sampling_zone,river,city,station,station_no,city_river,lat,lon,address
0,雙溪流域,雙溪,新北市,新寮大橋,1034,新北市雙溪,25.0317969,121.8347690,新北市雙溪區上林村北38號道
1,雙溪流域,雙溪,新北市,貢寮大橋,1035,新北市雙溪,25.0207100,121.9095289,新北市貢寮區102丙公路
2,得子口溪流域,得子口溪,宜蘭縣,竹安橋,1679,宜蘭縣得子口溪,24.8413333,121.8233611,宜蘭縣頭城鎮台2線北部濱海公路
3,得子口溪流域,得子口溪,宜蘭縣,美慶山莊,1332,宜蘭縣得子口溪,24.8314511,121.7470589,宜蘭縣礁溪鄉五峰路
4,得子口溪流域,得子口溪,宜蘭縣,七結橋(取代得子口橋),1221,宜蘭縣得子口溪,24.8031789,121.7651450,宜蘭縣礁溪鄉宜6鄉道(七結路)
...,...,...,...,...,...,...,...,...,...
331,None,None,None,None,None,"荖溪, Shoufeng, Hualien County, Taiwan Province,...",None,None,None
332,None,None,None,None,None,"壽豐溪(知亞干溪), Wanrong, Hualien County, Taiwan Pro...",None,None,None
333,None,None,None,None,None,"七家灣溪, Heping District, Taichung, 42413, Taiwan",None,None,None
334,None,None,None,None,None,南澳南溪,None,None,None


In [96]:
'''
fetch pollution sources from https://data.epa.gov.tw/api/v1/ems_p_01?limit=1000&api_key=9be7b239-557b-4c10-9775-78cadfc555e9&format=json and write to file
for each river stations get nearest pollution sources

https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude
'''
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))


url = 'https://data.epa.gov.tw/api/v1/ems_p_01?limit=85000&api_key=9be7b239-557b-4c10-9775-78cadfc555e9&format=json'
headers = {
    'Accept': 'application/json'}
r = requests.get(url, headers=headers)
data = r.content
data_dict = json.loads(data)
data_df = pd.DataFrame(data_dict['records'])
#data_df.groupby(['County']).count()

#taoyuan = data_df[data_df['County'] == '桃園市']
taoyuan['TWD97Lat'] = data_df['TWD97Lat'].replace('', np.nan)
taoyuan = taoyuan.dropna(subset=['TWD97Lat'])
taoyuan = taoyuan.astype({'TWD97Lat': 'float', 'TWD97Lon': 'float'})
locations_pollution_sources = taoyuan[['TWD97Lat', 'TWD97Lon']]
locations_pollution_sources = locations_pollution_sources.rename(columns={"TWD97Lat": "lat", "TWD97Lon": "lon" }).to_dict(orient='records')

river_stations = pd.read_json('final/river.json')

nearest_pollution_sources = []
for index, row in river_stations.iterrows():
    dict1 = {}
    coords = {'lat': row['lat'], 'lon': row['lon']}
    closest_src = closest(locations_pollution_sources, coords)
    src = taoyuan.loc[(taoyuan['TWD97Lat'] == closest_src['lat']) & (taoyuan['TWD97Lon'] == closest_src['lon'])]
    src_dict = src.to_dict(orient='records')[0]
    dict1.update(src_dict)
    
    if dict1 not in nearest_pollution_sources:
        nearest_pollution_sources.append(dict1)
    
print(nearest_pollution_sources[0])
print(nearest_pollution_sources[1])

with open('pollution_sources/pollution_sources.json', 'w', encoding='utf-8') as f:
    json.dump(nearest_pollution_sources, f, ensure_ascii=False, indent=4)



{'RegistrationNo': 'H48A6565', 'FacilityName': '巨甲子精密科技股份有限公司', 'County': '桃園市', 'Township': '龜山區', 'Address': '桃園市龜山區大華里頂湖三街１２號１樓', 'IndustryAreaName': '林口特定工業區(工四)', 'IndustryName': '金屬手工具及模具製造業', 'AirRegisteredStatus': '', 'WaterRegisteredStatus': '列管', 'WasteRegisteredStatus': '列管', 'ToxicRegisteredStatus': '', 'TWD97Lon': 121.361839010845, 'TWD97Lat': 24.9895696265992, 'UniformNo': '29109612'}
{'RegistrationNo': 'H4813192', 'FacilityName': '功泰通運有限公司', 'County': '桃園市', 'Township': '龜山區', 'Address': '桃園市龜山區中興里民權街４號１樓', 'IndustryAreaName': '', 'IndustryName': '汽車貨運業', 'AirRegisteredStatus': '', 'WaterRegisteredStatus': '', 'WasteRegisteredStatus': '列管', 'ToxicRegisteredStatus': '', 'TWD97Lon': 121.325578808837, 'TWD97Lat': 24.7604109446323, 'UniformNo': '89732296'}


In [7]:
'''
Using nominatim package to get river locations

https://nominatim.org/release-docs/develop/api/Search/#examples
https://nominatim.openstreetmap.org/search.php?q=%E6%96%B0%E7%AB%B9%E7%B8%A3%E9%B3%B3%E5%B1%B1%E6%BA%AA&polygon_geojson=1&viewbox=
https://nominatim.openstreetmap.org/search?q=%E6%96%B0%E7%AB%B9%E7%B8%A3%E9%B3%B3%E5%B1%B1%E6%BA%AA&format=json&polygon_geojson=1&addressdetails=1&limit=1
'''
geocoded_locations_nominatum = []
unique_locations_nominatum = []

    
# Create CSV file with unique city-river list
with open('rivers_geocoded_nominatum.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['location', 'name', 'type', 'lat', 'lon'])
    for location in locations['city_river'].unique():
        result = ''
        geo_json = {}
        resp = requests.get('https://nominatim.openstreetmap.org/search?q='+ location +'&format=json&polygon_geojson=1&addressdetails=1')
        if resp.status_code == 200:
            if resp.json():
                if location == '南港溪, Nantou County, Taiwan Province, 544, Taiwan':
                    result = resp.json()[1]
                elif location == '清水溪, Nantou County, Taiwan Province, 55759, Taiwan':
                    result = resp.json()[1]
                elif location == '谷溪':
                    result = resp.json()[2]                    
                else:
                    result = resp.json()[0]
                geo_json["type"] = "Feature"
                geo_json["geometry"] = result['geojson']
                geo_json["properties"] = {"location": location, "display": result['display_name'] }
                print(location, result['display_name'], result['type'], result['lat'], result['lon'])
                geocoded_locations_nominatum.append(geo_json)           
                spamwriter.writerow([location, result['display_name'], result['type'], result['lat'], result['lon']])
            else: 
                print(location + " not found")
                spamwriter.writerow([location, 'not found'])
        else:
            spamwriter.writerow([location, 'not found'])

# Create JSON file with geocoded data for each city-river  
geocoded_locations = {'data': geocoded_locations_nominatum}
with open('rivers_geocoded_nominatum.json', 'w', encoding='utf-8') as f:
    json.dump(geocoded_locations, f, ensure_ascii=False, sort_keys=True, indent=4)

新北市雙溪 雙溪, 雙溪區, 新北市, 22742, Taiwan river 25.0331832 121.85431
宜蘭縣得子口溪 得子口溪, 宜蘭縣, 臺灣省, 26244, Taiwan river 24.8154006 121.7671014
宜蘭縣蘭陽溪 蘭陽溪, 大同鄉, 宜蘭縣, 臺灣省, 267, Taiwan river 24.6098966 121.5327044
宜蘭縣冬山河 冬山河, 冬山鄉, 宜蘭縣, 臺灣省, 269, Taiwan river 24.6445902 121.7914189
宜蘭縣新城溪 新城溪, 宜蘭縣, 臺灣省, 26948, Taiwan river 24.594269 121.7927345
宜蘭縣蘇澳溪 蘇澳溪, 宜蘭縣, 臺灣省, 270, Taiwan river 24.573872 121.841303
宜蘭縣南澳溪 南奧溪, 宜蘭縣, 臺灣省, 27244, Taiwan river 24.5329943 121.7169998
宜蘭縣和平溪 和平溪, 宜蘭縣, 臺灣省, Taiwan river 24.3981148 121.5843567
花蓮縣立霧溪 立霧溪, 花蓮縣, 臺灣省, 972, Taiwan river 24.1782003 121.4765221
花蓮縣三棧溪 三棧溪, 花蓮縣, 臺灣省, 971, Taiwan river 24.0831072 121.5541238
花蓮縣美崙溪 美崙溪, 花蓮縣, 臺灣省, 97059, Taiwan river 24.0008125 121.5651216
吉安溪 (七腳川溪), Hualien County, Taiwan Province, 97342, Taiwan 吉安溪 (七腳川溪), 花蓮縣, 臺灣省, 97342, Taiwan stream 23.9828851 121.5703291
花蓮縣木瓜溪 木瓜溪, 秀林鄉, 花蓮縣, 臺灣省, 972, Taiwan river 23.9703325 121.4801381
花蓮縣花蓮溪 花蓮溪, 花蓮縣, 臺灣省, 979, Taiwan river 23.7461256 121.37623
花蓮縣秀姑巒溪 秀姑巒溪, 玉里鎮, 花蓮縣, 臺灣省, 981, Taiwan ri

七家灣溪, Heping District, Taichung, 42413, Taiwan 七家灣溪, 和平區, 臺中市, 42413, Taiwan stream 24.4067802 121.3142811
南澳南溪 南澳南溪, 南澳鄉, 宜蘭縣, 臺灣省, 272, Taiwan river 24.4247765 121.7355698
林尾溪 林尾溪, 礁溪鄉, 宜蘭縣, 臺灣省, 26244, Taiwan river 24.814497 121.7445166


In [ ]:
'''
- Use site code to link river 2019 data and API results so we can get the station lat and lon and plot on top of the rivers
- Map lat and lang back to river 2019 data so can be drawn on map

https://stackoverflow.com/questions/18175489/sending-soap-request-using-python-requests
https://wq.epa.gov.tw/WS_WQEPA/ws_wqepa.asmx/GetWQEPASampleData_River


url="https://wq.epa.gov.tw/WS_WQEPA/ws_wqepa.asmx?WSDL"
#resp = requests.get(url)

headers = {'content-type': 'text/xml; charset=utf-8', 
           'Host': 'wq.epa.gov.tw', 
           'SOAPAction': ''"http://localhost/WS_WQEpa/ws_wqepa/GetWaterBody_Station"''}


body = """<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <GetWaterBody_Station xmlns="http://localhost/WS_WQEpa/ws_wqepa">
      <waterbody_id>string</waterbody_id>
    </GetWaterBody_Station>
  </soap:Body>
</soap:Envelope>"""

response = requests.post(url,data=body,headers=headers)
print(response.content)
'''   

In [31]:
resp = requests.get('https://nominatim.openstreetmap.org/search?q=Taiping River, Tai''an Village, Beinan, Taitung County, Taiwan Province, 954, Taiwan&format=json&polygon_geojson=1&addressdetails=1')
resp.json()[0]

{'place_id': 273302736,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 762580977,
 'boundingbox': ['22.796856', '22.8191302', '121.0451577', '121.0951091'],
 'lat': '22.8066536',
 'lon': '121.0695349',
 'display_name': '太平溪, 泰安村, 卑南鄉, 臺東縣, 臺灣省, 954, Taiwan',
 'class': 'waterway',
 'type': 'river',
 'importance': 0.6050000000000001,
 'address': {'waterway': '太平溪',
  'city_district': '泰安村',
  'town': '卑南鄉',
  'county': '臺東縣',
  'state': '臺灣省',
  'postcode': '954',
  'country': 'Taiwan',
  'country_code': 'tw'},
 'geojson': {'type': 'LineString',
  'coordinates': [[121.0451577, 22.8191302],
   [121.0452908, 22.8182791],
   [121.0457713, 22.8174575],
   [121.0459797, 22.8169373],
   [121.0468248, 22.8162063],
   [121.0469753, 22.8158381],
   [121.0472879, 22.8152085],
   [121.0477423, 22.8149151],
   [121.0483501, 22.8143895],
   [121.049172, 22.814008],
   [121.0501445, 22.8139333],
   [121.0518174, 22.814024],
   [121.